In [53]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.special

In [54]:
iris = load_iris()
feature = pd.DataFrame(iris['data'],columns=iris['feature_names'])
target = pd.DataFrame(iris['target'],columns=['class'])

df = pd.concat([feature,target],axis = 1)

In [55]:
x = df.iloc[:,:-1].values
y = df.iloc[:,4].values
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=1)

In [91]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# use fit_transform to standardization
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [140]:
class MultiLayerNeuralNetwork:
    def __init__(self, layer_sizes, learning_rate):
        self.activation = lambda x: scipy.special.expit(x)
        self.layer_sizes = layer_sizes
        self.num_layers = len(layer_sizes)
        self.lr = learning_rate

        # Initialize weights
        self.weights = [np.random.normal(0.0, pow(layer_sizes[i], -0.5), (layer_sizes[i], layer_sizes[i - 1]))
                        for i in range(1, self.num_layers)]

    def train(self, inputs, targets):
        inputs = np.array(inputs, ndmin=2).T
        targets = np.array(targets, ndmin=2).T

        # Forward pass
        layer_outputs = [inputs]
        for i in range(self.num_layers - 2):
            layer_input = np.dot(self.weights[i], layer_outputs[-1])
            layer_output = self.activation(layer_input)
            layer_outputs.append(layer_output)

        final_input = np.dot(self.weights[-1], layer_outputs[-1])
        final_output = self.activation(final_input)
        layer_outputs.append(final_output)

        # Backward pass
        errors = [targets - final_output]
        for i in range(self.num_layers - 2, 0, -1):
            hidden_errors = np.dot(self.weights[i].T, errors[-1])
            errors.append(hidden_errors)

        # Update weights
        for i in range(self.num_layers - 2, -1, -1):
            self.weights[i] += self.lr * np.dot((errors[-(i + 1)] * layer_outputs[i + 1] * (1 - layer_outputs[i + 1])),
                                                np.transpose(layer_outputs[i]))

    def query(self, inputs):
        inputs = np.array(inputs, ndmin=2).T

        # Forward pass
        for i in range(self.num_layers - 2):
            layer_input = np.dot(self.weights[i], inputs)
            inputs = self.activation(layer_input)

        final_input = np.dot(self.weights[-1], inputs)
        final_output = self.activation(final_input)
        return final_output


In [141]:
nn2 = MultiLayerNeuralNetwork([4,100,10,3],0.01)

In [ ]:
epoche = 1000
performance = []
for e in range(epoche):
  scorecard = []
  for i, data in enumerate(X_train):
    input = data
    target = np.zeros(3)+0.01
    target[int(y_train[i])]  = 0.99
    nn2.train(input,target)
  for i, data in enumerate(X_test):
    Ans = int(y_test[i])
    input = data
    output = nn2.query(input)
    predict = np.argmax(output)
    if predict == Ans:
      scorecard.append(1)
    else:
      scorecard.append(0)
  scorecard_array = np.asarray(scorecard)
  performance.append(scorecard_array.sum()/scorecard_array.size)
plt.plot(performance)
plt.title('Neural Network Performance over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()